In [ ]:
!pip install torch transformers faiss-gpu sentence-transformers nltk requests beautifulsoup4 PyPDF2 pillow youtube-transcript-api
from IPython.display import clear_output
clear_output()

In [ ]:
import youtube_transcript_api
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
import requests
from bs4 import BeautifulSoup
import faiss
import nltk
import numpy as np

In [ ]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

youtube_url = "https://youtu.be/_BiAhjJIWx8?si=w1_t6spLHqbnWf6G"

def extract_video_id(youtube_url):
    parsed_url = urlparse(youtube_url)

    if 'youtube.com' in parsed_url.netloc:
        if 'v=' in parsed_url.query:
            return parse_qs(parsed_url.query)['v'][0]
        path_segments = parsed_url.path.split('/')
        return path_segments[path_segments.index('watch') + 1] if 'watch' in path_segments else None
    elif 'youtu.be' in parsed_url.netloc:
        return parsed_url.path[1:]
    else:
        return None

video_id = extract_video_id(youtube_url)
transcript = YouTubeTranscriptApi.get_transcript(video_id)
content = " ".join([entry['text'] for entry in transcript])

print("Transcript Text:", content)

Transcript Text: 3 tbsp Unsalted butter 3-4 Cloves of garlic (minced) A pinch of salt (optional) 1/2 tbsp Parsley Mix well 4 Slices of white bread Thank You My Friend Spread garlic butter evenly on one side I also can't miss The Crispiness of The Edges One more time~ Low heat Put the buttered side down Mozzarella cheese Gouda cheese (or Cheddar) Now put the buttered side up Cover with lid and cook until golden brown It's time to flip it over Good Crispiness~ Cover with lid and cook until golden brown You can also make it in the oven Bake in the oven at 190℃(374℉) for 10-12 minutes Delicious Cheese Garlic Sandwich is here! The combination of crispy garlic bread on the outside and cheese on the inside is delicious I'll NEVER Forget This Taste... See you next!


In [ ]:
llm = pipeline(model="databricks/dolly-v2-3b",
               torch_dtype=torch.bfloat16,
               trust_remote_code=True,
               device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def chunk_content_by_sentence(content):
    return sent_tokenize(content)

content_chunks = chunk_content_by_sentence(content)
print(f"Number of content chunks (sentences): {len(content_chunks)}")

Number of content chunks (sentences): 2


In [ ]:
chunk_embeddings = []
for chunk in content_chunks:
    inputs = tokenizer(chunk, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1).numpy()
    chunk_embeddings.append(embedding)

embeddings_np = np.vstack(chunk_embeddings)
print(f"Shape of embeddings: {embeddings_np.shape}")

Shape of embeddings: (2, 768)


In [ ]:
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)
print("Number of vectors in the FAISS index:", index.ntotal)

Number of vectors in the FAISS index: 2


In [ ]:
def generate_rag_response(query, model, tokenizer, index, content_chunks):

    query_inputs = tokenizer(query, return_tensors='pt')
    with torch.no_grad():
        query_embedding = model(**query_inputs).last_hidden_state.mean(dim=1).detach().numpy()
    k = 5
    distances, indices = index.search(query_embedding, k)
    relevant_contexts = [content_chunks[i] for i in indices[0]]
    combined_context = " ".join(relevant_contexts)

    input_text = (
    f"### Context Overview:\n"
    f"{combined_context}\n\n"
    f"### Instructions:\n"
    f"Using the provided context, please answer the following question. Your response should:\n"
    f"- Be as clear and concise as possible.\n"
    f"- Only answer the question based on the information in the context.\n"
    f"- If the answer is not present in the context, respond with 'I don't know' instead of providing an incorrect answer.\n"
    f"- Keep your answer no longer than three sentences.\n\n"
    f"### Question:\n"
    f"{query}\n\n"
    f"### Your Answer:"
    )

    response = llm(input_text, temperature=0.3)
    return response[0]['generated_text'].strip()

In [ ]:
query = "How long should I bake the sandwich?"
response = generate_rag_response(query, model, tokenizer, index, content_chunks)
print("Generated Response:", response)

Generated Response: You should bake the sandwich for about 10-12 minutes at low heat.
